In [ ]:
%%capture
import pandas as pd

## 루트 디렉토리 설정 

In [ ]:
# 작업 디렉토리 설정
path = r'D:\2020 구매기획팀\3. 정기업무\7. 포상 대상자 분석\파일 새로만들기\템플릿 업그레이드'

In [ ]:
# 실적 데이터
file_name = r'구매실적.xlsx'

df = pd.read_excel(path + '\\' + file_name , encoding='cp949')
df.head(1)

## 구매금액, 절감액, 절감률 

In [ ]:
if df['절감액'].item == 0:
    df['절감률'] = 0
else:
    df['절감률'] = df['절감액']/(df['구매금액'] + df['절감액'])

## 목표달성률  

### 원료 

+ 목표 > 0: 

<blockquote>목표 달성률 = 1+ {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

+ 목표 < 0: 

<blockquote>목표 달성률 = {(절감액 - 도전목표) / abs(도전목표)}</blockquote>

### 설비

+ 설비/외자: 절감률 / 7%
+ 순환: 제외 

### 인테리어

+ 인테리어: 절감률 / 4.5%

In [ ]:
df.loc[df['도전목표'] < 0, '목표 달성률'] = (df['절감액'] - df['도전목표']) / abs(df['도전목표'])
df.loc[df['도전목표'] == 0, '목표 달성률'] = df['목표 달성률'] = 0
df.loc[df['도전목표'] > 0, '목표 달성률'] = 1 + \
    ((df['절감액'] - df['도전목표']) / abs(df['도전목표']))
df.loc[df['구분'] == '설비', '목표 달성률'] = df['절감률'] / 0.07
df.loc[df['구분'] == '외자', '목표 달성률'] = df['절감률'] / 0.07 #인수인계는 5.5%였으나, 김재욱 차장님이 7% 라고 하심
df.loc[df['구분'] == '인테리어', '목표 달성률'] = df['절감률'] / 0.045

In [ ]:
df['index'] = df.loc[(df['구분']!='순환')].groupby('담당자')['구매금액'].rank(method='first', ascending=False)
df_achieved_rate = df[['팀','담당자', '구분','구매금액','절감액','도전목표','절감률','목표 달성률', 'index']]
df_achieved_rate.loc[(df_achieved_rate['구분']!='순환') & (df_achieved_rate['index'] <2)]

## 신규아이디어 제안 성과 분석

### 신규절감제안 절감률 
+ 월 구매금액 대비 신규 절감 아이디어 제안 금액의 절감률
+ 신규아이디어 제안서 상정주차 기준으로 포상 기준월 담당자별 절감액 산출

In [ ]:
# 신규아이디어 절감액 데이터 import
file_name2 = r'아이디어절감액.xlsx'
df2 = pd.read_excel(path + '\\' + file_name2 , encoding='cp949')
df2.head(1)

In [ ]:
df_sum = df.groupby('담당자')['구매금액'].sum()
df_sum = pd.DataFrame(df_sum)
df_sum.head(1)

In [ ]:
df_merge = pd.merge(df_sum, df2, how='inner', on='담당자')
df_merge['제안절감률'] = df_merge['아이디어절감액'] / df_merge['구매금액']
df_idea = df_merge
df_idea.head(5)

### 신규절감제안 달성률
+ 신규 아이디어 제안의 당월 목표 대비 절감 달성률
+ 원가절감 Tracking 목표 및 절감액 

<blockquote>신규절감제안 달성률 = 당월 제안절감액 / 당월 제안목표</blockquote>

In [ ]:
# 원가절감 Tracking 데이터 import
file_name3 = r'원가절감Tracking.xlsx'
df3 = pd.read_excel(path + '\\' + file_name3 , encoding='cp949')
df3.head(1)

In [ ]:
df3['제안달성률'] = df3['제안절감액'] / df3['제안목표']
df_idea_achieved = df3
df_idea_achieved.head(5)

## 상품개발 성과 분석

### 상품개발 목표 대비 달성률 & 3개월 매출 평균  

+ 상품개발 달성률: 신규 상품개발의 당월 목표 대비 매출 달성률
<blockquote> 상품개발 달성률 = 당월 매출/ 상품개발목표 </blockquote>

+ 상품개발 매출액 평균: 최근 3개월 매출 평균금액 (단, 1월, 2월은 해당 월 까지의 누계 평균 반영)

In [ ]:
# 원가절감 Tracking 데이터 import
file_name4 = r'상품개발실적.xlsx'
df4 = pd.read_excel(path + '\\' + file_name4 , encoding='cp949')
df4.head(1)

In [ ]:
def divide_by_zero(a, b):
    if b.item == 0:
        return 0.0
    elif a.item == 0:
        return 0.0
    else:
        return a / b

df4['매출목표달성률'] = divide_by_zero(df4['당월매출'], df4['상품개발목표'])
df4